### Import Library

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Load Data

In [ ]:

# 데이터 로드 
df = pd.read_csv('/Users/jisupark_1/workspace/star_track_python/PRJ_Meteo/dataset/train_heat.csv')  # 실제 파일명으로 변경
# 컬럼명 정리 (train_heat. 접두사 제거)
df.columns = [col.replace('train_heat.', '') if 'train_heat.' in col else col for col in df.columns]


### Data Exploration

In [4]:

print("=" * 60)
print("📊 기본 데이터 정보")
print("=" * 60)

# 1. 기본 정보
print(f"데이터 형태: {df.shape}")
print(f"컬럼 수: {len(df.columns)}")
print(f"행 수: {len(df)}")
print()

# 2. 컬럼 정보
print("컬럼 목록:")
print(df.columns.tolist())
print()

# 3. 데이터 타입
print("데이터 타입:")
print(df.dtypes)
print()

# 4. 기본 통계
print("기본 통계:")
print(df.describe())
print()

print("=" * 60)
print("🔍 결측치 분석")
print("=" * 60)

# 5. 결측치 현황 (전체)
print("전체 결측치 현황:")
missing_info = df.isnull().sum()
missing_percent = (df.isnull().sum() / len(df)) * 100
missing_df = pd.DataFrame({
    '결측치_개수': missing_info,
    '결측치_비율(%)': missing_percent
})
print(missing_df[missing_df['결측치_개수'] > 0])
print()

# 6. -99 값 분석 (결측치로 표시된 값들)
print("-99로 표시된 결측치 분석:")
for col in df.columns:
    if df[col].dtype in ['int64', 'float64']:
        count_99 = (df[col] == -99).sum()
        if count_99 > 0:
            percent_99 = (count_99 / len(df)) * 100
            print(f"{col}: {count_99}개 ({percent_99:.2f}%)")
print()

# 7. 지사별 결측치 패턴
print("지사별 결측치 패턴:")
for branch in sorted(df['branch_id'].unique()):
    branch_data = df[df['branch_id'] == branch]
    print(f"\n지사 {branch} (총 {len(branch_data)}개 레코드):")
    
    for col in ['si', 'hm', 'ta', 'wd', 'ws', 'rn_day', 'rn_hr1', 'ta_chi']:
        if col in df.columns:
            # 일반 결측치
            missing_count = branch_data[col].isnull().sum()
            # -99 결측치  
            neg99_count = (branch_data[col] == -99).sum()
            total_missing = missing_count + neg99_count
            
            if total_missing > 0:
                missing_percent = (total_missing / len(branch_data)) * 100
                print(f"  {col}: {total_missing}개 ({missing_percent:.1f}%) [일반결측:{missing_count}, -99:{neg99_count}]")

print()

print("=" * 60)
print("📅 시간 데이터 분석")
print("=" * 60)

# 8. 시간 데이터 파싱 및 분석
df['datetime'] = pd.to_datetime(df['tm'], format='%Y%m%d%H', errors='coerce')
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day
df['hour'] = df['datetime'].dt.hour

print("연도별 데이터 분포:")
print(df['year'].value_counts().sort_index())
print()

print("월별 데이터 분포:")
print(df['month'].value_counts().sort_index())
print()

print("시간별 데이터 분포:")
print(df['hour'].value_counts().sort_index())
print()

print("지사별 데이터 분포:")
print(df['branch_id'].value_counts().sort_index())
print()

print("=" * 60)
print("🌡️ 기상 데이터 범위 분석")
print("=" * 60)

# 9. 기상 변수 범위 확인
weather_vars = ['ta', 'wd', 'ws', 'rn_day', 'rn_hr1', 'hm', 'si', 'ta_chi']

for var in weather_vars:
    if var in df.columns:
        # -99 제외하고 분석
        clean_data = df[df[var] != -99][var].dropna()
        if len(clean_data) > 0:
            print(f"{var}:")
            print(f"  범위: {clean_data.min():.2f} ~ {clean_data.max():.2f}")
            print(f"  평균: {clean_data.mean():.2f}")
            print(f"  중앙값: {clean_data.median():.2f}")
            
            # 이상치 의심 값들
            Q1 = clean_data.quantile(0.25)
            Q3 = clean_data.quantile(0.75)
            IQR = Q3 - Q1
            outlier_low = Q1 - 1.5 * IQR
            outlier_high = Q3 + 1.5 * IQR
            outliers = clean_data[(clean_data < outlier_low) | (clean_data > outlier_high)]
            
            if len(outliers) > 0:
                print(f"  이상치 의심: {len(outliers)}개 ({len(outliers)/len(clean_data)*100:.1f}%)")
            print()

print("=" * 60)
print("🔥 열수요 데이터 분석")
print("=" * 60)

# 10. 열수요 기본 통계
print("열수요 기본 통계:")
print(f"범위: {df['heat_demand'].min():.1f} ~ {df['heat_demand'].max():.1f} Gcal/h")
print(f"평균: {df['heat_demand'].mean():.1f} Gcal/h")
print(f"중앙값: {df['heat_demand'].median():.1f} Gcal/h")
print(f"표준편차: {df['heat_demand'].std():.1f} Gcal/h")
print()

# 11. 지사별 열수요 특성
print("지사별 열수요 특성:")
branch_stats = df.groupby('branch_id')['heat_demand'].agg([
    'count', 'mean', 'std', 'min', 'max'
]).round(1)
print(branch_stats)
print()

# 12. 월별 열수요 패턴
print("월별 평균 열수요:")
monthly_demand = df.groupby('month')['heat_demand'].mean().round(1)
print(monthly_demand)
print()

# 13. 시간별 열수요 패턴
print("시간별 평균 열수요:")
hourly_demand = df.groupby('hour')['heat_demand'].mean().round(1)
print(hourly_demand)
print()

print("=" * 60)
print("🔗 상관관계 분석")
print("=" * 60)

# 14. 기상변수와 열수요 간 상관관계
print("기상변수와 열수요 간 상관관계:")
correlation_vars = ['ta', 'wd', 'ws', 'rn_day', 'rn_hr1', 'hm', 'ta_chi', 'heat_demand']
available_vars = [var for var in correlation_vars if var in df.columns]

# -99 값을 NaN으로 처리 후 상관관계 계산
df_corr = df[available_vars].replace(-99, np.nan)
correlations = df_corr.corr()['heat_demand'].drop('heat_demand').sort_values(key=abs, ascending=False)
print(correlations.round(3))
print()

print("=" * 60)
print("📊 데이터 품질 체크")
print("=" * 60)

# 15. 중복 데이터 확인
duplicates = df.duplicated(subset=['tm', 'branch_id']).sum()
print(f"중복 레코드: {duplicates}개")

# 16. 시계열 연속성 확인
print("\n시계열 연속성 확인 (각 지사별):")
for branch in sorted(df['branch_id'].unique()):
    branch_data = df[df['branch_id'] == branch].sort_values('tm')
    if len(branch_data) > 1:
        # 시간 간격 확인
        branch_data['time_diff'] = branch_data['datetime'].diff()
        expected_interval = pd.Timedelta(hours=1)
        irregular_intervals = (branch_data['time_diff'] != expected_interval).sum() - 1  # 첫 번째는 항상 NaN
        print(f"지사 {branch}: {irregular_intervals}개 불규칙 간격")

print("\n✅ 데이터 탐색 완료!")
print("\n다음 단계를 위한 요약:")
print("- 주요 결측치가 있는 변수들을 확인했습니다")
print("- 각 지사별 데이터 특성을 파악했습니다")  
print("- 기상변수와 열수요 간 상관관계를 확인했습니다")
print("- 이제 결측치 처리 방법을 결정할 수 있습니다")

📊 기본 데이터 정보
데이터 형태: (499301, 12)
컬럼 수: 12
행 수: 499301

컬럼 목록:
['Unnamed: 0', 'tm', 'branch_id', 'ta', 'wd', 'ws', 'rn_day', 'rn_hr1', 'hm', 'si', 'ta_chi', 'heat_demand']

데이터 타입:
Unnamed: 0       int64
tm               int64
branch_id       object
ta             float64
wd             float64
ws             float64
rn_day         float64
rn_hr1         float64
hm             float64
si             float64
ta_chi         float64
heat_demand      int64
dtype: object

기본 통계:
          Unnamed: 0            tm             ta             wd  \
count  499301.000000  4.993010e+05  499301.000000  499301.000000   
mean   249651.000000  2.022067e+09      10.621007     194.322872   
std    144135.927716  8.172155e+05      20.878917     118.348853   
min         1.000000  2.021010e+09     -99.000000     -99.000000   
25%    124826.000000  2.021100e+09       4.400000     105.000000   
50%    249651.000000  2.022070e+09      14.200000     212.100000   
75%    374476.000000  2.023040e+09      22.500